In [ ]:
import numpy as np

import matplotlib.pyplot as pl
import matplotlib as mpl


mpl.rc('figure', figsize=(10,8))

'''
mpl.rc('figure', figsize=(14,12))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'


pl.style.use('fivethirtyeight')
'''

font = {'size'   : 13}

mpl.rc('font', **font)

#mpl.rcParams['lines.linewidth']=4
#mpl.rcParams['axes.labelsize']=16


import os
import re

from scipy.io import loadmat
from scipy.ndimage import gaussian_filter1d

In [ ]:
data_folder='../../Data/SP-2021_01_12-Q394_fmaskedCAP_data'

listFiles = os.listdir(data_folder)

listFilesMat=[]

for filename in listFiles:
    if re.match('.*.mat', filename):
        listFilesMat.append(filename)
listFilesMat.sort()

### Functions

In [ ]:
#filter ind if necessary
begin_ind=395   #27
end_ind=2000  #49

In [ ]:
def split_name(filename):
    '''returns (picNumber, masker filename)'''
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', filename)
    assert m, f'{filename} did not match'
    p, name= m.groups()
    return int(p), name

map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    p, name = split_name(fln)
    if name not in map_table:
        map_table[name]=[p]
    else:
        li=map_table[name]
        li.append(p)
        map_table[name]=li
        
filtered_map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    p, name = split_name(fln)
    if p>=begin_ind and p<=end_ind:

        if name not in filtered_map_table:
            filtered_map_table[name]=[p]
        else:
            li=filtered_map_table[name]
            li.append(p)
            filtered_map_table[name]=li
    
    

In [ ]:
def loadPic(n):
    found=False
    for filename in listFilesMat:
        if f'p{str(n).zfill(4)}' in filename:
            found=True
            break
    assert found, f'pic {n} not found.'
    arr= loadmat(f'{data_folder}/{filename}')
    return arr['data_struct']

def get_info_pic(pic):
    #avg
    arr=pic['AD_Data'][0][0]['AD_Avg_V'][0][0][0]
    
    #HACK only C or R
    '''all_data=pic['AD_Data'][0][0]['AD_All_V'][0][0]
    arr=np.zeros_like(all_data[0])
    for i in range(len(all_data)):
        if i%2==0:
            arr+=all_data[i]
    arr/=len(all_data)/2
    '''
    
    return {'arr':arr,
            'XstartPlot_ms':pic['Stimuli'][0][0]['CAP_intervals'][0][0]['XstartPlot_ms'][0][0][0],
            'XendPlot_ms': pic['Stimuli'][0][0]['CAP_intervals'][0][0]['XendPlot_ms'][0][0][0],
           'name':pic['Stimuli'][0][0]['masker'][0][0]['name'][0][0]}


    


In [ ]:
struct=loadPic(7)
struct_data=struct['AD_Data'][0][0]['AD_Avg_V'][0][0][0]

struct_data=struct['Stimuli'][0][0]['CAP_intervals'][0][0]['XstartPlot_ms'][0][0][0]

struct_data=struct['Stimuli'][0][0]['CAP_intervals'][0][0]['XendPlot_ms'][0][0][0]


struct_data=struct['Stimuli'][0][0]['masker'][0][0]['name'][0][0]


struct_data=struct['AD_Data'][0][0]['AD_All_V'][0][0]



struct_data=struct['Stimuli'][0][0]['masker'][0][0]['bands'][0][0]['fc_low']


print(struct_data)


In [ ]:

def plotPic(picDic=None, n=None, label='', tmin=0, baselineSig=None):
    '''
    Plot pic given picture dictionary or pic number. 
    Args:
        baseline: Picture dictionary for baseline signal'''
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    pic_info=get_info_pic(picDic)
    if label=='':
        label=pic_info['name']
        
    
    val=pic_info['arr']
    
    t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))
    if baselineSig is not None:
        val=val-baselineSig
    #val=np.cumsum(val) #HACK!!
    pl.plot(t, val, label=label)
    pl.xlim([tmin,  pic_info['XendPlot_ms']])
    
def getMaskerSig(dic, name):
    
    picNums=dic[name]
    firstPic=True
    for picNum in picNums:
        picDic=loadPic(picNum)
        
    
        pic_info=get_info_pic(picDic)
    
        if firstPic:
            #val=np.sum(picDic['valAll'][1::2], axis=0)
            val=pic_info['arr']
            firstPic=False
        else:
            #val+=np.sum(picDic['valAll'][1::2], axis=0)
            val+=pic_info['arr']
    val/=len(picNums)
    return val
    
def plotMasker(dic, name, tmin=0, tmax=12, baseline=None):
    '''
    Plot avg signal for masker given masker name and dictionary of picNums 
    Args:
        baseline: signal for baseline signal'''
    label=name
    val=getMaskerSig(dic, name)
    
    picNums=dic[name]
    for picNum in picNums:
        picDic=loadPic(picNum)
        pic_info=get_info_pic(picDic)
        
        break

    t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))
    if baseline is not None:
        val=val-baseline
    #val=gaussian_filter1d(val, 1.5) #HACK
    #val=np.cumsum(val) #HACK!!
    #val-=val[300] #HACK!!
    pl.plot(t, val, label=label)
    if tmax==0:
        tmax=pic_info['XendPlot_ms']
    pl.xlim([tmin, tmax])
    pl.xlabel('t (ms)')
    pl.ylabel('Amplitude')
    
'''
def getMaskerAtten(picDic=None, n=None):
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    ans=picDic['masker_atten_dB']
    return ans[0][0]
'''

### Show examples of signals

In [ ]:
dic=filtered_map_table

In [ ]:
names=list(dic.keys())
names.sort()
#print(names)

In [ ]:
broadband=getMaskerSig(dic, 'broadband_noise')
nomasker=getMaskerSig(dic, 'nomasker')

In [ ]:
broadband_sig=getMaskerSig(dic, 'broadband_noise')+getMaskerSig(dic, '2_broadband_noise')+getMaskerSig(dic, '3_broadband_noise')
broadband_sig/=3

nomasker_sig=getMaskerSig(dic, 'nomasker')+getMaskerSig(dic, '2_nomasker')+getMaskerSig(dic, '3_nomasker')
nomasker_sig/=3

In [ ]:
plotMasker(dic, 'nomasker')
plotMasker(dic, '2_nomasker')
plotMasker(dic, '3_nomasker')

In [ ]:
info=get_info_pic(loadPic(800))
XstartPlot_ms=info['XstartPlot_ms']

XendPlot_ms=info['XendPlot_ms']
t=np.linspace(XstartPlot_ms, XendPlot_ms, num=len(broadband_sig))
plotMasker(dic, 'broadband_noise')


plotMasker(dic, '2_broadband_noise')

plotMasker(dic, '3_broadband_noise')


pl.plot(t, broadband_sig, label='broadband average')


#pl.figure()
plotMasker(dic, 'nomasker')
plotMasker(dic, '2_nomasker')
plotMasker(dic, '3_nomasker')

pl.plot(t, nomasker_sig, label='no masker avg')

#pl.xlim([0, 7])
#pl.ylim([-0.01, 0.01])

pl.legend()

### Narrow-band analysis

In [ ]:
#plotMasker(dic, 'nomasker')
plotMasker(dic, '1_hp_10000Hz')
plotMasker(dic, '2_hp_9000Hz')
plotMasker(dic, '3_hp_8000Hz')
plotMasker(dic, '4_hp_7000Hz')
plotMasker(dic, '5_hp_6000Hz')
plotMasker(dic, '6_hp_5000Hz')
plotMasker(dic, '7_hp_4000Hz')
plotMasker(dic, '8_hp_3200Hz')
plotMasker(dic, '9_hp_2400Hz')
plotMasker(dic, '10_hp_1800Hz')
plotMasker(dic, '11_hp_1500Hz')
plotMasker(dic, '12_hp_1200Hz')
plotMasker(dic, 'broadband_noise')
pl.xlim([5,12])
pl.legend(loc='upper left')

In [ ]:
#plotMasker(dic, 'nomasker')
plotMasker(dic, '1_hp_10000Hz', baseline=broadband_sig)
plotMasker(dic, '2_hp_9000Hz', baseline=broadband_sig)
plotMasker(dic, '3_hp_8000Hz', baseline=broadband_sig)
plotMasker(dic, '4_hp_7000Hz', baseline=broadband_sig)
plotMasker(dic, '5_hp_6000Hz', baseline=broadband_sig)
plotMasker(dic, '6_hp_5000Hz', baseline=broadband_sig)
plotMasker(dic, '7_hp_4000Hz', baseline=broadband_sig)
plotMasker(dic, '8_hp_3200Hz', baseline=broadband_sig)
plotMasker(dic, '9_hp_2400Hz', baseline=broadband_sig)
plotMasker(dic, '10_hp_1800Hz', baseline=broadband_sig)
plotMasker(dic, '11_hp_1500Hz', baseline=broadband_sig)
plotMasker(dic, '12_hp_1200Hz', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')  #5 dB higher
pl.xlim([4.5,10.4])
pl.legend()
#pl.savefig('narrow-band.svg')

In [ ]:
#plotMasker(dic, 'nomasker')
'''plotMasker(dic, '1_hp_10000Hz', baseline=getMaskerSig(dic, '2_hp_9000Hz'))
plotMasker(dic, '2_hp_9000Hz', baseline=getMaskerSig(dic, '3_hp_8000Hz'))
plotMasker(dic, '3_hp_8000Hz', baseline=getMaskerSig(dic, '4_hp_7000Hz'))
plotMasker(dic, '4_hp_7000Hz', baseline=getMaskerSig(dic, '5_hp_6000Hz'))
plotMasker(dic, '5_hp_6000Hz', baseline=getMaskerSig(dic, '6_hp_5000Hz'))
plotMasker(dic, '6_hp_5000Hz', baseline=getMaskerSig(dic, '7_hp_4000Hz'))'''

plotMasker(dic, '7_hp_4000Hz', baseline=getMaskerSig(dic, '8_hp_3200Hz'))
plotMasker(dic, '8_hp_3200Hz', baseline=getMaskerSig(dic, '9_hp_2400Hz'))
plotMasker(dic, '9_hp_2400Hz', baseline=getMaskerSig(dic, '10_hp_1800Hz'))
plotMasker(dic, '10_hp_1800Hz', baseline=getMaskerSig(dic, '11_hp_1500Hz'))
plotMasker(dic, '11_hp_1500Hz', baseline=getMaskerSig(dic, '12_hp_1200Hz'))
plotMasker(dic, '12_hp_1200Hz', baseline=broadband_sig)

#plotMasker(dic, 'broadband_noise')  
pl.legend()

In [ ]:
#plotMasker(dic, 'nomasker')
#plotMasker(dic, '1_hp_10000Hz', baseline=getMaskerSig(dic, '3_hp_8000Hz'))
plotMasker(dic, '2_hp_9000Hz', baseline=getMaskerSig(dic, '4_hp_7000Hz'))
#plotMasker(dic, '3_hp_8000Hz', baseline=getMaskerSig(dic, '5_hp_6000Hz'))
plotMasker(dic, '4_hp_7000Hz', baseline=getMaskerSig(dic, '6_hp_5000Hz'))
#plotMasker(dic, '5_hp_6000Hz', baseline=getMaskerSig(dic, '7_hp_4000Hz'))
plotMasker(dic, '6_hp_5000Hz', baseline=getMaskerSig(dic, '8_hp_3200Hz'))
#plotMasker(dic, '7_hp_4000Hz', baseline=getMaskerSig(dic, '9_hp_2400Hz'))
plotMasker(dic, '8_hp_3200Hz', baseline=getMaskerSig(dic, '10_hp_1800Hz'))
#plotMasker(dic, '9_hp_2400Hz', baseline=getMaskerSig(dic, '11_hp_1500Hz'))
plotMasker(dic, '10_hp_1800Hz', baseline=getMaskerSig(dic, '12_hp_1200Hz'))
#plotMasker(dic, '11_hp_1500Hz', baseline=baseline=broadband_sig)
plotMasker(dic, '12_hp_1200Hz', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')  #5 dB higher
pl.legend()

### Notch 8k

In [ ]:
plotMasker(dic, '1_notch8000_bw2300_60dB')
plotMasker(dic, '2_notch8000_bw2300_55dB')
plotMasker(dic, '3_notch8000_bw2300_50dB')
plotMasker(dic, '4_notch8000_bw2300_45dB')
plotMasker(dic, '5_notch8000_bw2300_40dB')
plotMasker(dic, '6_notch8000_bw2300_35dB')
plotMasker(dic, '7_notch8000_bw2300_30dB')
plotMasker(dic, '8_notch8000_bw2300_25dB')
plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
plotMasker(dic, '1_notch8000_bw2300_60dB', baseline=broadband_sig)
plotMasker(dic, '2_notch8000_bw2300_55dB', baseline=broadband_sig)
plotMasker(dic, '3_notch8000_bw2300_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch8000_bw2300_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch8000_bw2300_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch8000_bw2300_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch8000_bw2300_30dB', baseline=broadband_sig)
plotMasker(dic, '8_notch8000_bw2300_25dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=['1_notch8000_bw2300_60dB',
'2_notch8000_bw2300_55dB',
'3_notch8000_bw2300_50dB',
'4_notch8000_bw2300_45dB',
'5_notch8000_bw2300_40dB',
'6_notch8000_bw2300_35dB',
'7_notch8000_bw2300_30dB',
'8_notch8000_bw2300_25dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,30,25]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

### Notch 6k

In [ ]:
plotMasker(dic, '1_notch6000_bw2000_60dB')
plotMasker(dic, '2_notch6000_bw2000_55dB')
plotMasker(dic, '3_notch6000_bw2000_50dB')
plotMasker(dic, '4_notch6000_bw2000_45dB')
plotMasker(dic, '5_notch6000_bw2000_40dB')
plotMasker(dic, '6_notch6000_bw2000_35dB')
plotMasker(dic, '7_notch6000_bw2000_32dB')
plotMasker(dic, '8_notch6000_bw2000_29dB')
plotMasker(dic, '9_notch6000_bw2000_26dB')
plotMasker(dic, '10_notch6000_bw2000_23dB')
plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
plotMasker(dic, '1_notch6000_bw2000_60dB', baseline=broadband_sig)
plotMasker(dic, '2_notch6000_bw2000_55dB', baseline=broadband_sig)
plotMasker(dic, '3_notch6000_bw2000_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch6000_bw2000_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch6000_bw2000_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch6000_bw2000_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch6000_bw2000_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch6000_bw2000_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch6000_bw2000_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch6000_bw2000_23dB', baseline=broadband_sig)
pl.plot(t, getMaskerSig(dic, 'broadband_noise')-getMaskerSig(dic, '2_broadband_noise'), label='noise floor')
pl.legend()

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=['1_notch6000_bw2000_60dB',
'2_notch6000_bw2000_55dB',
'3_notch6000_bw2000_50dB',
'4_notch6000_bw2000_45dB',
'5_notch6000_bw2000_40dB',
'6_notch6000_bw2000_35dB',
'7_notch6000_bw2000_32dB',
'8_notch6000_bw2000_29dB',
'9_notch6000_bw2000_26dB',
'10_notch6000_bw2000_23dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,32,29,26,23]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
'''1-2bands6000-5200-bw800-6850-bw600.json
1-narrorband6100_bw200_20dB.json
1-notch6000_bw2000_60dB.json
2-2bands6000-5200-bw600-6850-bw600.json
2-narrorband6000_bw400_20dB.json
2-notch6000_bw1000_32dB.json
2-notch6000_bw2000_55dB.json
3-2bands6000-5200-bw600-6660-bw981.json
3-narrorband6000_bw600_29dB.json
3-notch6000_bw2000_50dB.json
3-notch6300_bw1200_26dB.json
4-narrorband5800_bw500_32dB.json
4-notch5800_bw1200_20dB.json
4-notch6000_bw2000_45dB.json
5-narrorband6000_bw800_32dB.json
5-notch5800_bw800_24dB.json
5-notch6000_bw2000_40dB.json
6-notch6000_bw900_29dB.json
6-notch6000_bw2000_35dB.json
7-notch6000_bw1100_32dB.json
7-notch6000_bw2000_32dB.json
8-notch5700_bw800_35dB.json
8-notch6000_bw2000_29dB.json
9-notch6000_bw2000_26dB.json
9-notch6100_bw1000_23dB.json
10-notch6000_bw2000_23dB.json
'''

plotMasker(dic, '1_narrorband6100_bw200_20dB',  baseline=nomasker_sig)

plotMasker(dic, '2_narrorband6000_bw400_20dB', baseline=nomasker_sig)

plotMasker(dic, '3_narrorband6000_bw600_29dB',  baseline=nomasker_sig)


plotMasker(dic, '4_narrorband5800_bw500_32dB',  baseline=nomasker_sig)

plotMasker(dic, '5_narrorband6000_bw800_32dB',  baseline=nomasker_sig)

plotMasker(dic, '3_2bands6000_5200_bw600_6660_bw981',  baseline=nomasker_sig)
pl.legend()

In [ ]:
plotMasker(dic, '4_notch5800_bw1200_20dB', baseline=broadband_sig)
plotMasker(dic, '3_notch6300_bw1200_26dB', baseline=broadband_sig)
plotMasker(dic, '5_notch5800_bw800_24dB', baseline=broadband_sig)
plotMasker(dic, '9_notch6100_bw1000_23dB', baseline=broadband_sig)

#sig=getMaskerSig(dic, '3_notch6300_bw1200_26dB')
#maskingSig=nomasker_sig-broadband_sig

pl.legend()

### Notch 5k

In [ ]:
plotMasker(dic, '1_notch5000_bw2000_60dB')
plotMasker(dic, '2_notch5000_bw2000_55dB')
plotMasker(dic, '3_notch5000_bw2000_50dB')
plotMasker(dic, '4_notch5000_bw2000_45dB')
plotMasker(dic, '5_notch5000_bw2000_40dB')
plotMasker(dic, '6_notch5000_bw2000_35dB')
plotMasker(dic, '7_notch5000_bw2000_32dB')
plotMasker(dic, '8_notch5000_bw2000_29dB')
plotMasker(dic, '9_notch5000_bw2000_26dB')
plotMasker(dic, '10_notch5000_bw2000_23dB')
plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
plotMasker(dic, '1_notch5000_bw2000_60dB', baseline=broadband_sig)
plotMasker(dic, '2_notch5000_bw2000_55dB', baseline=broadband_sig)
plotMasker(dic, '3_notch5000_bw2000_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch5000_bw2000_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch5000_bw2000_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch5000_bw2000_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch5000_bw2000_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch5000_bw2000_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch5000_bw2000_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch5000_bw2000_23dB', baseline=broadband_sig)
pl.xlim([4.5,10.4])
pl.legend()
#pl.savefig('notch5000.svg')

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=['1_notch5000_bw2000_60dB',
'2_notch5000_bw2000_55dB',
'3_notch5000_bw2000_50dB',
'4_notch5000_bw2000_45dB',
'5_notch5000_bw2000_40dB',
'6_notch5000_bw2000_35dB',
'7_notch5000_bw2000_32dB',
'8_notch5000_bw2000_29dB',
'9_notch5000_bw2000_26dB',
'10_notch5000_bw2000_23dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,32,29,26,23]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
'''1-2bands5000-4300-bw600-5600-bw600.json
1-narrorband5000_bw300_20dB.json
1-notch5000_bw1000_31dB.json
1-notch5000_bw2000_60dB.json
2-2bands5000-4250-bw700-5750-bw700.json
2-narrorband4800_bw200_22dB.json
2-notch5000_bw2000_55dB.json
2-notch5200_bw900_34dB.json
3-2bands5000-4350-bw500-5700-bw800.json
3-narrorband4800_bw500_31dB.json
3-notch4800_bw1100_26dB.json
3-notch5000_bw2000_50dB.json
4-2bands5000-4250-bw800-5850-bw400.json
4-narrorband5200_bw700_31dB.json
4-notch4800_bw1300_20dB.json
4-notch5000_bw2000_45dB.json
5-narrorband5300_bw800_33dB.json
5-notch4800_bw700_27dB.json
5-notch5000_bw2000_40dB.json
6-notch5000_bw900_27dB.json
6-notch5000_bw2000_35dB.json
7-notch5000_bw2000_32dB.json
7-notch5300_bw800_20dB.json
8-notch5000_bw2000_29dB.json
9-notch5000_bw2000_26dB.json
10-notch5000_bw2000_23dB.json
'''

plotMasker(dic, '1_narrorband5000_bw300_20dB',  baseline=nomasker_sig)

plotMasker(dic, '2_narrorband4800_bw200_22dB', baseline=nomasker_sig)

plotMasker(dic, '3_narrorband4800_bw500_31dB',  baseline=nomasker_sig)


plotMasker(dic, '5_narrorband5300_bw800_33dB',  baseline=nomasker_sig)

plotMasker(dic, '4_narrorband5200_bw700_31dB',  baseline=nomasker_sig)

plotMasker(dic, '4_2bands5000_4250_bw800_5850_bw400',  baseline=nomasker_sig)
pl.legend()

In [ ]:
plotMasker(dic, '4_notch4800_bw1300_20dB',  baseline=broadband_sig)
plotMasker(dic, '7_notch5300_bw800_20dB', baseline=broadband_sig)

#plotMasker(dic, '1_notch5000_bw1000_31dB',  baseline=broadband_sig)
#plotMasker(dic, '2_notch5200_bw900_34dB',  baseline=broadband_sig)

### Notch 4k

I don't have the varying notch amplitude bc of mistake

In [ ]:
plotMasker(dic, '4_notch3800_1000_21dB', baseline=broadband_sig)
plotMasker(dic, '5_notch3800_1200_20dB', baseline=broadband_sig)
plotMasker(dic, '8_notch4200_bw1100_20dB', baseline=broadband_sig)
pl.legend()

### Notch 3k

In [ ]:
plotMasker(dic, '1_notch3000_bw1500_60dB')
plotMasker(dic, '2_notch3000_bw1500_55dB')
plotMasker(dic, '4_notch3000_bw1500_50dB')
plotMasker(dic, '5_notch3000_bw1500_45dB')
plotMasker(dic, '6_notch3000_bw1500_40dB')
plotMasker(dic, '7_notch3000_bw1500_35dB')
plotMasker(dic, '8_notch3000_bw1500_32dB')
plotMasker(dic, '9_notch3000_bw1500_29dB')
plotMasker(dic, '10_notch3000_bw1500_26dB')
plotMasker(dic, '11_notch3000_bw1500_23dB')
plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
plotMasker(dic, '1_notch3000_bw1500_60dB', baseline=broadband_sig)  #pb on this one?
plotMasker(dic, '2_notch3000_bw1500_55dB', baseline=broadband_sig)
plotMasker(dic, '4_notch3000_bw1500_50dB', baseline=broadband_sig)
plotMasker(dic, '5_notch3000_bw1500_45dB', baseline=broadband_sig)
plotMasker(dic, '6_notch3000_bw1500_40dB', baseline=broadband_sig)
plotMasker(dic, '7_notch3000_bw1500_35dB', baseline=broadband_sig)
plotMasker(dic, '8_notch3000_bw1500_32dB', baseline=broadband_sig)
plotMasker(dic, '9_notch3000_bw1500_29dB', baseline=broadband_sig)
plotMasker(dic, '10_notch3000_bw1500_26dB', baseline=broadband_sig)
plotMasker(dic, '11_notch3000_bw1500_23dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=['1_notch3000_bw1500_60dB'
,'2_notch3000_bw1500_55dB'
,'4_notch3000_bw1500_50dB'
,'5_notch3000_bw1500_45dB'
,'6_notch3000_bw1500_40dB'
,'7_notch3000_bw1500_35dB'
,'8_notch3000_bw1500_32dB'
,'9_notch3000_bw1500_29dB'
,'10_notch3000_bw1500_26dB'
,'11_notch3000_bw1500_23dB']
    
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,32,29,26,23]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

Notch 3k-bis

In [ ]:
plotMasker(dic, '1_notch3000_bw1700_60dB')
plotMasker(dic, '2_notch3000_bw1700_55dB')
plotMasker(dic, '3_notch3000_bw1700_50dB')
plotMasker(dic, '4_notch3000_bw1700_45dB')
plotMasker(dic, '5_notch3000_bw1700_40dB')
plotMasker(dic, '6_notch3000_bw1700_35dB')
plotMasker(dic, '7_notch3000_bw1700_32dB')
plotMasker(dic, '8_notch3000_bw1700_29dB')
plotMasker(dic, '9_notch3000_bw1700_26dB')
plotMasker(dic, '10_notch3000_bw1700_23dB')
plotMasker(dic, 'broadband_noise')

pl.legend()

In [ ]:
pl.plot(t, getMaskerSig(dic, 'broadband_noise'))
pl.plot(t, getMaskerSig(dic, '1_notch3000_bw1700_60dB'))
pl.axhline()
pl.plot(t, getMaskerSig(dic, '1_notch3000_bw1700_60dB')-getMaskerSig(dic, 'broadband_noise'))

In [ ]:
plotMasker(dic, '1_notch3000_bw1700_60dB', baseline=broadband_sig)

In [ ]:
plotMasker(dic, '1_notch3000_bw1700_60dB', baseline=broadband_sig)
plotMasker(dic, '2_notch3000_bw1700_55dB', baseline=broadband_sig)
plotMasker(dic, '3_notch3000_bw1700_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch3000_bw1700_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch3000_bw1700_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch3000_bw1700_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch3000_bw1700_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch3000_bw1700_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch3000_bw1700_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch3000_bw1700_23dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise', baseline=broadband_sig)
pl.legend()

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=['1_notch3000_bw1700_60dB',
'2_notch3000_bw1700_55dB',
'3_notch3000_bw1700_50dB',
'4_notch3000_bw1700_45dB',
'5_notch3000_bw1700_40dB',
'6_notch3000_bw1700_35dB',
'7_notch3000_bw1700_32dB',
'8_notch3000_bw1700_29dB',
'9_notch3000_bw1700_26dB',
'10_notch3000_bw1700_23dB']

for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,32,29,26,23]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')


In [ ]:
'''1-2bands-3000-2500-bw400-3600-bw400.json
1-narrorband2900_bw200_21dB.json
1-notch3000_1000_34dB.json
1-notch3000_bw1500_60dB.json
2-2bands3000-2550-bw300-3500-bw400.json
2-narrorband3200_bw300_21dB.json
2-notch3000_500_21dB.json
2-notch3000_bw1500_55dB.json
3-narrorband3000_bw500_27dB.json
3-notch2800_600_21dB.json
4-narrorband3000_bw700_36dB.json
4-notch2800_800_26dB.json
4-notch3000_bw1500_50dB.json
5-notch2900_700_26dB.json
5-notch3000_bw1500_45dB.json
6-notch3000_700_35dB.json
6-notch3000_bw1500_40dB.json
7-notch3000_900_25dB.json
7-notch3000_bw1500_35dB.json
8-notch3000_1100_25dB.json
8-notch3000_bw1500_32dB.json
9-notch3000_bw1500_29dB.json
9-notch3200_700_30dB.json
10-notch3000_bw1500_26dB.json
11-notch3000_bw1500_23dB.json
'''

plotMasker(dic, '2_notch3000_500_21dB',  baseline=broadband_sig)

plotMasker(dic, '4_notch2800_800_26dB',  baseline=broadband_sig)

plotMasker(dic, '5_notch2900_700_26dB', baseline=broadband_sig)
plotMasker(dic, '8_notch3000_1100_25dB', baseline=broadband_sig)
#plotMasker(dic, 'nomasker', baseline=broadband_sig)
pl.legend()


### notch 2.2k

In [ ]:
plotMasker(dic, '1_notch2200_bw1500_60dB')
plotMasker(dic, '2_notch2200_bw1500_55dB')
plotMasker(dic, '3_notch2200_bw1500_50dB')
plotMasker(dic, '4_notch2200_bw1500_45dB')
plotMasker(dic, '5_notch2200_bw1500_40dB')
plotMasker(dic, '6_notch2200_bw1500_35dB')
plotMasker(dic, '7_notch2200_bw1500_32dB')
plotMasker(dic, '8_notch2200_bw1500_29dB')
plotMasker(dic, '9_notch2200_bw1500_26dB')
plotMasker(dic, '10_notch2200_bw1500_23dB')
plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
plotMasker(dic, '1_notch2200_bw1500_60dB', baseline=broadband_sig)
plotMasker(dic, '2_notch2200_bw1500_55dB', baseline=broadband_sig)
plotMasker(dic, '3_notch2200_bw1500_50dB', baseline=broadband_sig)
plotMasker(dic, '4_notch2200_bw1500_45dB', baseline=broadband_sig)
plotMasker(dic, '5_notch2200_bw1500_40dB', baseline=broadband_sig)
plotMasker(dic, '6_notch2200_bw1500_35dB', baseline=broadband_sig)
plotMasker(dic, '7_notch2200_bw1500_32dB', baseline=broadband_sig)
plotMasker(dic, '8_notch2200_bw1500_29dB', baseline=broadband_sig)
plotMasker(dic, '9_notch2200_bw1500_26dB', baseline=broadband_sig)
plotMasker(dic, '10_notch2200_bw1500_23dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=['1_notch2200_bw1500_60dB',
'2_notch2200_bw1500_55dB',
'3_notch2200_bw1500_50dB',
'4_notch2200_bw1500_45dB',
'5_notch2200_bw1500_40dB',
'6_notch2200_bw1500_35dB',
'7_notch2200_bw1500_32dB',
'8_notch2200_bw1500_29dB',
'9_notch2200_bw1500_26dB',
'10_notch2200_bw1500_23dB']
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,32,29,26,23]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
'''1-2bands-2200-1850-bw300-2650-bw300.json
1-narrorband2100_bw500_36dB.json
1-notch2200_500_20dB.json
1-notch2200_bw1500_60dB.json
2-2bands-2200-1750-bw500-2600-bw400.json
2-narrorband2100_bw400_29dB.json
2-notch2200_bw1500_55dB.json
2-notch2300_700_20dB.json
3-narrorband2300_bw300_24dB.json
3-notch2200_bw1500_50dB.json
3-notch2300_700_24dB.json
4-narrorband2200_bw200_21dB.json
4-notch2200_bw1500_45dB.json
4-notch2400_800_24dB.json
5-notch2200_bw1500_40dB.json
5-notch2300_900_31dB.json
6-notch2200_bw1500_35dB.json
6-notch2300_400_31dB.json
7-notch2000_400_29dB.json
7-notch2200_bw1500_32dB.json
8-notch2100_700_34dB.json
8-notch2200_bw1500_29dB.json
9-notch2100_1000_34dB.json
9-notch2200_bw1500_26dB.json
10-notch2200_bw1500_23dB.json'''


plotMasker(dic, '1_notch2200_500_20dB',  baseline=broadband_sig)



plotMasker(dic, '2_notch2300_700_20dB',  baseline=broadband_sig)


plotMasker(dic, '3_notch2300_700_24dB',  baseline=broadband_sig)
pl.legend()


### Notch 1.5k

In [ ]:
plotMasker(dic, '30_notch1500_bw1000_60dB')
plotMasker(dic, '31_notch1500_bw1000_55dB')
plotMasker(dic, '32_notch1500_bw1000_50dB')
plotMasker(dic, '33_notch1500_bw1000_45dB')
plotMasker(dic, '34_notch1500_bw1000_40dB')
plotMasker(dic, '35_notch1500_bw1000_35dB')
plotMasker(dic, '36_notch1500_bw1000_32dB')
plotMasker(dic, '37_notch1500_bw1000_29dB')
plotMasker(dic, '38_notch1500_bw1000_26dB')
plotMasker(dic, '39_notch1500_bw1000_23dB')
plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
plotMasker(dic, '30_notch1500_bw1000_60dB', baseline=broadband_sig)
plotMasker(dic, '31_notch1500_bw1000_55dB', baseline=broadband_sig)
plotMasker(dic, '32_notch1500_bw1000_50dB', baseline=broadband_sig)
plotMasker(dic, '33_notch1500_bw1000_45dB', baseline=broadband_sig)
plotMasker(dic, '34_notch1500_bw1000_40dB', baseline=broadband_sig)
plotMasker(dic, '35_notch1500_bw1000_35dB', baseline=broadband_sig)
plotMasker(dic, '36_notch1500_bw1000_32dB', baseline=broadband_sig)
plotMasker(dic, '37_notch1500_bw1000_29dB', baseline=broadband_sig)
plotMasker(dic, '38_notch1500_bw1000_26dB', baseline=broadband_sig)
plotMasker(dic, '39_notch1500_bw1000_23dB', baseline=broadband_sig)
#plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]


masker_list=['30_notch1500_bw1000_60dB',
'31_notch1500_bw1000_55dB',
'32_notch1500_bw1000_50dB',
'33_notch1500_bw1000_45dB',
'34_notch1500_bw1000_40dB',
'35_notch1500_bw1000_35dB',
'36_notch1500_bw1000_32dB',
'37_notch1500_bw1000_29dB',
'38_notch1500_bw1000_26dB',
'39_notch1500_bw1000_23dB']

for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    broadband_sig_trunc=broadband_sig[300:]
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    rms.append(np.std(sig-broadband_sig_trunc))
pl.plot(-np.array([60,55, 50, 45,40,35,32,29,26,23]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation (re:max)')
pl.ylabel('Amplitude')

In [ ]:
'''1-2bands-1500-1200-bw400-2100-400.json
1-narrorband1600_bw200_20dB.json
1-notch1500_bw400_20dB.json
2-2bands-1500-1150-bw300-1967-423.json
2-narrorband1600_bw300_31dB.json
2-notch1500_bw600_20dB.json
3-narrorband1600_bw500_36dB.json
3-notch1500_bw800_20dB.json
4-notch1400_bw300_30dB.json
5-notch1400_bw500_30dB.json
6-notch1600_bw700_30dB.json
7-notch1600_bw600_35dB.json
8-notch1700_bw800_25dB.json
30-notch1500_bw1000_60dB.json
31-notch1500_bw1000_55dB.json
32-notch1500_bw1000_50dB.json
33-notch1500_bw1000_45dB.json
34-notch1500_bw1000_40dB.json
35-notch1500_bw1000_35dB.json
36-notch1500_bw1000_32dB.json
37-notch1500_bw1000_29dB.json
38-notch1500_bw1000_26dB.json
39-notch1500_bw1000_23dB.json'''








plotMasker(dic, '1_narrorband1600_bw200_20dB',  baseline=nomasker_sig)

plotMasker(dic, '2_narrorband1600_bw300_31dB', baseline=nomasker_sig)

plotMasker(dic, '3_narrorband1600_bw500_36dB',  baseline=nomasker_sig)


plotMasker(dic, '1_2bands_1500_1200_bw400_2100_400',  baseline=nomasker_sig)
pl.legend()

In [ ]:
plotMasker(dic, '1_notch1500_bw400_20dB',  baseline=broadband_sig)

plotMasker(dic, '2_notch1500_bw600_20dB',  baseline=broadband_sig)

plotMasker(dic, '3_notch1500_bw800_20dB',  baseline=broadband_sig)

plotMasker(dic, '6_notch1600_bw700_30dB',  baseline=broadband_sig)
plotMasker(dic, '8_notch1700_bw800_25dB', baseline=broadband_sig)

#plotMasker(dic, 'nomasker', baseline=broadband_sig)
pl.legend()


### lp

In [ ]:
plotMasker(dic, '1_lp_1500Hz')
plotMasker(dic, '2_lp_2000Hz')
plotMasker(dic, '3_lp_3000Hz')
plotMasker(dic, '4_lp_4000Hz')
plotMasker(dic, '5_lp_5000Hz')
plotMasker(dic, '6_lp_6000Hz')
plotMasker(dic, '7_lp_7000Hz')
plotMasker(dic, '8_lp_8000Hz')
pl.legend()

In [ ]:
plotMasker(dic, '1_lp_1500Hz', baseline=nomasker_sig)
plotMasker(dic, '2_lp_2000Hz', baseline=nomasker_sig)
plotMasker(dic, '3_lp_3000Hz', baseline=nomasker_sig)
plotMasker(dic, '4_lp_4000Hz', baseline=nomasker_sig)
plotMasker(dic, '5_lp_5000Hz', baseline=nomasker_sig)
plotMasker(dic, '6_lp_6000Hz', baseline=nomasker_sig)
plotMasker(dic, '7_lp_7000Hz', baseline=nomasker_sig)
plotMasker(dic, '8_lp_8000Hz', baseline=nomasker_sig)
pl.legend()

In [ ]:
plotMasker(dic, '1_lp_1500Hz', baseline=getMaskerSig(dic, '3_lp_3000Hz'))
#plotMasker(dic, '2_lp_2000Hz', baseline=getMaskerSig(dic, '4_lp_4000Hz')
plotMasker(dic, '3_lp_3000Hz', baseline=getMaskerSig(dic, '5_lp_5000Hz'))
#plotMasker(dic, '4_lp_4000Hz', baseline=getMaskerSig(dic, '6_lp_6000Hz')
plotMasker(dic, '5_lp_5000Hz', baseline=getMaskerSig(dic, '7_lp_7000Hz'))
plotMasker(dic, '6_lp_6000Hz', baseline=getMaskerSig(dic, '8_lp_8000Hz'))
#plotMasker(dic, '7_lp_7000Hz', baseline=getMaskerSig(dic, '3_lp_3000Hz')
#plotMasker(dic, '8_lp_8000Hz', baseline=getMaskerSig(dic, '3_lp_3000Hz')
pl.legend()

In [ ]:
plotMasker(dic, '4_lp_2500Hz', baseline=nomasker)
plotMasker(dic, '6_lp_4000Hz',  baseline=nomasker)
plotMasker(dic, '7_lp_5000Hz',  baseline=nomasker)
plotMasker(dic, '10_lp_7000Hz',  baseline=nomasker)
pl.legend()

### Click delay

In [ ]:
clickdelay_map_table={'8ms':[923,924], '10ms':[913,914], '14ms': [917,918], '12ms': [919,920]}  

In [ ]:
for delay in clickdelay_map_table:
    plotMasker(clickdelay_map_table, delay)
plotMasker(dic, 'nomasker')
pl.legend()

### Narrow 5k

In [ ]:
#plotMasker(dic, '1_narrow5k_2k')  #-20db add. attenuation...
plotMasker(dic, '1_narrow5k_bw1700_attn20dB')
plotMasker(dic, '2_narrow5k_bw1400_attn20dB')
plotMasker(dic, '3_narrow5k_bw1100_attn20dB')
pl.legend()

In [ ]:
#plotMasker(dic, '1_narrow5k_2k', baseline=nomasker) #-20db add. attenuation...
plotMasker(dic, '1_narrow5k_bw1700_attn20dB', baseline=nomasker)
plotMasker(dic, '2_narrow5k_bw1400_attn20dB', baseline=nomasker)
plotMasker(dic, '3_narrow5k_bw1100_attn20dB', baseline=nomasker)
plotMasker(dic, '4_narrow5k_bw900_attn20dB', baseline=nomasker)
pl.legend()

### Notches 5k

Too much attenuation

### Narrow around 5k

In [ ]:

plotMasker(dic, '4_hp_6000Hz')
plotMasker(dic, '5_hp_5000Hz')
plotMasker(dic, '6_hp_4000Hz')

plotMasker(dic, '1_hp_5500Hz')
plotMasker(dic, '5_hp_4500Hz')
pl.legend()

In [ ]:
plotMasker(dic, '4_hp_6000Hz', baseline=broadband)

plotMasker(dic, '1_hp_5500Hz', baseline=broadband)
plotMasker(dic, '5_hp_5000Hz', baseline=broadband)

plotMasker(dic, '5_hp_4500Hz', baseline=broadband)
plotMasker(dic, '6_hp_4000Hz', baseline=broadband)

pl.legend()